In [22]:
import pandas as pd

# # Helper function to load CSV with flexible row limits
def load_data(file, column_names, max_rows=1000, sep=r'$'):
    try:
        df = pd.read_csv(file, sep=sep, header=None, names=column_names, nrows=max_rows)
    except Exception as e:
        print(f"Error loading {file}: {e}")
        df = pd.DataFrame(columns=column_names)  # Return an empty DataFrame if error occurs
    return df

# Load data from each .txt file, limiting to first 1000 rows if applicable

# Adverse Reaction Report (limit to 1000 rows)
df_report = load_data('reports.txt', 
                      ['REPORT_ID', 'REPORT_NO', 'VERSION_NO', 'DATRECEIVED', 'DATINTRECEIVED', 'MAH_NO', 
                       'REPORT_TYPE_CODE', 'REPORT_TYPE_ENG', 'REPORT_TYPE_FR', 'GENDER_CODE', 
                       'GENDER_ENG', 'GENDER_FR', 'AGE', 'AGE_Y', 'AGE_UNIT_ENG', 'AGE_UNIT_FR', 
                       'OUTCOME_CODE', 'OUTCOME_ENG', 'OUTCOME_FR', 'WEIGHT', 'WEIGHT_UNIT_ENG', 
                       'WEIGHT_UNIT_FR', 'HEIGHT', 'HEIGHT_UNIT_ENG', 'HEIGHT_UNIT_FR', 
                       'SERIOUSNESS_CODE', 'SERIOUSNESS_ENG', 'SERIOUSNESS_FR', 'DEATH', 
                       'DISABILITY', 'CONGENITAL_ANOMALY', 'LIFE_THREATENING', 'HOSP_REQUIRED', 
                       'OTHER_MEDICALLY_IMP_COND', 'REPORTER_TYPE_ENG', 'REPORTER_TYPE_FR', 
                       'SOURCE_CODE', 'SOURCE_ENG', 'SOURCE_FR', 'E2B_IMP_SAFETYREPORT_ID', 
                       'AUTHORITY_NUMB', 'COMPANY_NUMB'])

# Drug Product (limit to 1000 rows)
df_drug = load_data('drug_products.txt', 
                    ['DRUG_PRODUCT_ID', 'DRUGNAME'])

# Drug Product Ingredients (limit to 1000 rows)
df_ingredients = load_data('drug_product_ingredients.txt', 
                           ['DRUG_PRODUCT_INGREDIENT_ID', 'DRUG_PRODUCT_ID', 'DRUGNAME', 
                            'ACTIVE_INGREDIENT_ID', 'ACTIVE_INGREDIENT_NAME'])

# Reactions (limit to 1000 rows)
df_reaction = load_data('reactions.txt', 
                        ['REACTION_ID', 'REPORT_ID', 'DURATION', 'DURATION_UNIT_ENG', 'DURATION_UNIT_FR', 
                         'PT_NAME_ENG', 'PT_NAME_FR', 'SOC_NAME_ENG', 'SOC_NAME_FR', 'MEDDRA_VERSION'])

# Outcome (limit to 1000 rows)
df_outcome = load_data('outcome_lx.txt', 
                       ['OUTCOME_LX_ID', 'OUTCOME_CODE', 'OUTCOME_EN', 'OUTCOME_FR'])

# Report Drug (limit to 1000 rows)
df_report_drug = load_data('report_drug.txt', 
                           ['REPORT_DRUG_ID', 'REPORT_ID', 'DRUG_PRODUCT_ID', 'DRUGNAME', 'DRUGINVOLV_ENG', 
                            'DRUGINVOLV_FR', 'ROUTEADMIN_ENG', 'ROUTEADMIN_FR', 'UNIT_DOSE_QTY', 
                            'DOSE_UNIT_ENG', 'DOSE_UNIT_FR', 'FREQUENCY', 'FREQ_TIME', 
                            'FREQUENCY_TIME_ENG', 'FREQUENCY_TIME_FR', 'FREQ_TIME_UNIT_ENG', 
                            'FREQ_TIME_UNIT_FR', 'THERAPY_DURATION', 'THERAPY_DURATION_UNIT_ENG', 
                            'THERAPY_DURATION_UNIT_FR', 'DOSAGEFORM_ENG', 'DOSAGEFORM_FR'])

# Report Drug Indication (limit to 1000 rows)
df_report_drug_indication = load_data('report_drug_indication.txt', 
                                      ['REPORT_DRUG_ID', 'REPORT_ID', 'DRUG_PRODUCT_ID', 'DRUGNAME', 
                                       'INDICATION_NAME_ENG', 'INDICATION_NAME_FR'])



In [24]:
# # 1. Merge Report with Report_Drug on REPORT_ID
df_combined = pd.merge(df_report, df_report_drug, on='REPORT_ID', how='left')

# # 2. Merge the combined DataFrame with Drug_Product on DRUG_PRODUCT_ID
df_combined = pd.merge(df_combined, df_drug, on='DRUG_PRODUCT_ID', how='left')

# # 3. Merge with Drug_Product_Ingredients on DRUG_PRODUCT_ID
df_combined = pd.merge(df_combined, df_ingredients, on='DRUG_PRODUCT_ID', how='left')

# # 4. Merge with Reactions on REPORT_ID
df_combined = pd.merge(df_combined, df_reaction, on='REPORT_ID', how='left')

# # 5. Merge with Outcome_LX on OUTCOME_CODE
df_combined = pd.merge(df_combined, df_outcome, on='OUTCOME_CODE', how='left')

# # 6. Merge with Report_Drug_Indication on REPORT_ID and DRUG_PRODUCT_ID
# df_report_drug_indication.drop(columns=['DRUGNAME'], inplace=True)  # Drop DRUGNAME column to avoid duplicate columns
df_report_drug_indication_without_drugname = df_report_drug_indication.drop(columns=['DRUGNAME'], inplace=False)
df_combined = pd.merge(df_combined, df_report_drug_indication_without_drugname, on=['REPORT_ID', 'DRUG_PRODUCT_ID'], how='left')

# # Now you have a combined DataFrame with all the relevant data
print(df_combined.head())  # Display the first few rows of the combined DataFrame
print(df_combined.columns)  # Display the columns of the combined DataFrame

   REPORT_ID  REPORT_NO  VERSION_NO DATRECEIVED DATINTRECEIVED  MAH_NO  \
0          1          1           0   05-JUN-73      05-JUN-73     NaN   
1          1          1           0   05-JUN-73      05-JUN-73     NaN   
2          2          2           0   05-JUN-73      05-JUN-73     NaN   
3          3          3           0   05-JUN-73      05-JUN-73     NaN   
4          3          3           0   05-JUN-73      05-JUN-73     NaN   

   REPORT_TYPE_CODE REPORT_TYPE_ENG         REPORT_TYPE_FR  GENDER_CODE  ...  \
0                 7     Spontaneous  Déclaration spontanée          2.0  ...   
1                 7     Spontaneous  Déclaration spontanée          2.0  ...   
2                 7     Spontaneous  Déclaration spontanée          2.0  ...   
3                 7     Spontaneous  Déclaration spontanée          1.0  ...   
4                 7     Spontaneous  Déclaration spontanée          1.0  ...   

             PT_NAME_FR                                       SOC_NAME_ENG

In [26]:
df_combined.to_csv('combined_data.csv', index=False)  # Save the combined DataFrame to a CSV file